# Goal
- Train GPT2 on wiki text

## Steps
- Read, download data
- Train tokenizer
- Prepare sliding window data loader
- Use GPT2 model
- Use train/test loop

### Read, download data

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
val_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="validation")
test_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

# filter out empty lines
train_dataset = train_dataset.filter(lambda x: len(x["text"]) > 0)
val_dataset = val_dataset.filter(lambda x: len(x["text"]) > 0)
test_dataset = test_dataset.filter(lambda x: len(x["text"]) > 0)

train_dataset

In [ ]:
type(train_dataset['text'][1])

### Train tokenizer

In [ ]:
from utils import get_train_tokenizer
wrapped_tokenizer = get_train_tokenizer(train_dataset, vocab_size=10000)

In [ ]:
wrapped_tokenizer("<|endoftext|>")['input_ids']

In [ ]:
wrapped_tokenizer(["Hello my name is Ajay"])['input_ids']

### Prepare sliding window data loader

In [ ]:
import torch 
from utils import slide_window

tokenized_train_dataset = train_dataset.map(slide_window, batched=True, fn_kwargs={"wrapped_tokenizer": wrapped_tokenizer, "max_length": 256})
tokenized_val_dataset = val_dataset.map(slide_window, batched=True, fn_kwargs={"wrapped_tokenizer": wrapped_tokenizer, "max_length": 256})
tokenized_test_dataset = test_dataset.map(slide_window, batched=True, fn_kwargs={"wrapped_tokenizer": wrapped_tokenizer, "max_length": 256})


tokenized_train_dataset


In [ ]:
tokenized_train_dataset[1]

In [ ]:
import torch 
from datasets import Dataset as HFDataset
from torch.utils.data import Dataset

class HuggingFaceDataset(Dataset):
    """
    Wraps a Hugging Face Dataset to be used with a PyTorch DataLoader.

    Assumes the Hugging Face dataset has 'input' and 'target' columns.
    """

    def __init__(self, hf_dataset: HFDataset):
        self.hf_dataset = hf_dataset

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        item = self.hf_dataset[idx]
        return item['input_ids'], item['output_ids'], item['attention_mask']

def collate_fn(batch):
    input_ids = [item[0] for item in batch]
    output_ids = [item[1] for item in batch]
    attention_mask = [item[2] for item in batch]

    

    # set dtype to long

    input_ids_list = torch.tensor(input_ids, dtype=torch.long)
    output_ids_list = torch.tensor(output_ids, dtype=torch.long)
    attention_mask_list = torch.tensor(attention_mask, dtype=torch.long)
    return input_ids_list, output_ids_list, attention_mask_list

batch_size = 20
train_torch_dataset = HuggingFaceDataset(tokenized_train_dataset)
val_torch_dataset = HuggingFaceDataset(tokenized_val_dataset)
test_torch_dataset = HuggingFaceDataset(tokenized_test_dataset)

train_torch_dataloader = torch.utils.data.DataLoader(
    train_torch_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)
val_torch_dataloader = torch.utils.data.DataLoader(
    val_torch_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn
)
test_torch_dataloader = torch.utils.data.DataLoader(
    test_torch_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn
)

train_torch_dataloader

In [ ]:
batch = next(iter(train_torch_dataloader)) # (input_ids, output_ids)
input_ids, output_ids, attention_mask = batch
input_ids.shape, output_ids.shape, attention_mask.shape

### Use GPT2 model

In [ ]:
from models import GPT2

num_train_batches = tokenized_train_dataset.num_rows // batch_size
num_val_batches = tokenized_val_dataset.num_rows // batch_size

config = {
        "emb_dim": 768,
        "heads": 12,
        "layers": 12,
        "vocab_size": 50257,
        "context_length": 1024,
        "device": torch.device("cuda"),
        "drop_out": 0.1,
        "train_test_split": 0.8,
        "num_epochs": 25,
        "model_path": "../model_files/gpt2.pth",
        "num_train_batches" : num_train_batches,
        "num_train_batches" : num_train_batches,
        "learning_rate" : 1e-4,
        "num_test_batches" : num_val_batches,
    }

gpt2 = GPT2(config)
gpt2.to(config['device'])
gpt2

### Use train/test loop

In [ ]:
from utils import train

train(gpt2, train_torch_dataloader, val_torch_dataloader, config, use_fp_16=True)

### Generate text

In [ ]:
dir(wrapped_tokenizer)

In [ ]:
tokenized = wrapped_tokenizer("Hello my name is", truncation=True, max_length=100, padding="max_length", return_tensors="pt")

attention_mask = tokenized['attention_mask'].to(config["device"])
input_ids = tokenized['input_ids'].to(config["device"])

print(attention_mask)
print(input_ids)

prediction = gpt2(input_ids)
next_token = prediction.argmax(dim=-1)

print(prediction.shape)

In [ ]:

def generate_text(starting_text, model, tokenizer, config, num_output_tokens=100):
    device = config["device"]
    output_tokens = []

    input_encoding = tokenizer(starting_text, truncation=True, max_length=1024, padding="max_length", return_tensors="pt")
    input_ids = input_encoding['input_ids'].to(device)
    
    output_text = f"{starting_text} -> "
    for _ in range(num_output_tokens):
        
        next_token_logits = model(input_ids)[:,-1,:]
        next_token = next_token_logits.argmax(dim=-1)

        output_tokens.append(next_token.item())

        next_token = next_token.to(device)
        next_token_decoded = tokenizer.decode(next_token.item())
        output_text += next_token_decoded
        

        # Append the predicted token to the input for the next iteration
        input_ids = torch.cat((input_ids, next_token.unsqueeze(0)), dim=1)
        input_ids = input_ids[:, -1024:]

        if next_token.item() == tokenizer.eos_token_id:
            break
        
        
        

        



        #output_text += next_text
    print(output_text)

generate_text("The capital is", gpt2, wrapped_tokenizer, config)


In [ ]:
wrapped_tokenizer.encode("Who is the president of the United States?")